In [ ]:
import geopandas as gpd
import momepy as mm
import matplotlib.pyplot as plt
import requests
import pygeos as pyg
import momepy as mm
from networkx.algorithms.centrality import betweenness_centrality_subset
import numpy as np

In [ ]:
from AutoCordon.manipulate_geometries import overlay_gdf_with_geom

In [ ]:
roads_path = r"..\tests\data\SU_open_roads_small.geojson"
roads = gpd.read_file(roads_path)
roads = roads.explode().reset_index(drop=True)
# roads_simplified = mm.remove_false_nodes(roads)

In [ ]:
centre_ll = (-1.406425284433567, 50.91263710769894)
distance = 170
distance_max = 320

In [ ]:
coords_api = fr"http://webapps.bgs.ac.uk/data/webservices/CoordConvert_LL_BNG.cfc?method=LatLongToBNG&lat={centre_ll[1]}&lon={centre_ll[0]}"
response = requests.get(coords_api).json()
centre = (response["EASTING"], response["NORTHING"])

In [ ]:
point = pyg.points(centre)

hole = pyg.buffer(point, distance)
hole_ring = pyg.get_exterior_ring(hole)

shell = pyg.buffer(point, distance_max)
shell_ring = pyg.get_exterior_ring(shell)

shell_points = overlay_gdf_with_geom(roads, shell_ring)
max_nodes = list(zip(shell_points.geometry.x, shell_points.geometry.y))

hole_points = overlay_gdf_with_geom(roads, hole_ring)
inner_nodes = list(zip(hole_points.geometry.x, hole_points.geometry.y))

donut = pyg.polygons(shell_ring, holes=[hole_ring])
donut_roads = overlay_gdf_with_geom(roads, donut)
# donut_roads_simplified = mm.remove_false_nodes(donut_roads)

graph = mm.gdf_to_nx(donut_roads)

In [ ]:
donut_roads = overlay_gdf_with_geom(roads, donut)
hole_roads = overlay_gdf_with_geom(roads, hole)
# graph = mm.gdf_to_nx(roads_circle)
# n = (n for n in range(len(graph.nodes)))
# node_list = list(graph.nodes)

In [ ]:
# f, ax =  plt.subplots(figsize=(15, 15))
# roads_circle.plot(ax=ax)
# node = node_list[next(n)]
# ax.scatter(*node[:2], color="r")
# print(node)

In [ ]:
centre_node = (441823.0, 112745.0, 0.0)
shell_nodes = list(zip(shell_points.geometry.x, shell_points.geometry.y, [0.0] * len(shell_points)))
hole_nodes = list(zip(hole_points.geometry.x, hole_points.geometry.y, [0.0] * len(hole_points)))

hole_gdf = gpd.GeoDataFrame({"geometry": [hole], "id": ["id1"]})
shell_gdf = gpd.GeoDataFrame({"geometry": [shell], "id": ["id1"]})

axis_buffer = distance_max + 20
xlim = (centre[0] - axis_buffer, centre[0] + axis_buffer)
ylim = (centre[1] - axis_buffer, centre[1] + axis_buffer)

In [ ]:
graph = mm.gdf_to_nx(donut_roads)

removals = []
for i in range(9):
    f, ax = plt.subplots(1, 2, figsize=(15, 10), sharex=True, sharey=True)
    
    betweeness = betweenness_centrality_subset(graph, hole_nodes, shell_nodes)
#     betweeness = {node: betweeness[node] for node in graph.nodes}
    max_node, max_betweeness = max(list(betweeness.items()), key=lambda x: x[1])
    
    betweeness_values = [(key[0], key[1], value) for key, value in betweeness.items()]
    betweeness_values = sorted(betweeness_values, key=lambda x: x[2])
    x, y, c = zip(*betweeness_values)
    
    for a in ax:
        roads.plot(ax=a, color="lightgrey", zorder=1)
        shell_gdf.plot(ax=a, color="#a09ad5", alpha=0.2, zorder=2)
        hole_gdf.plot(ax=a, color="#D40058", alpha=0.2, zorder=3)
        hole_roads.plot(ax=a, color="#D40058", alpha=0.4, zorder=4)
        a.set_xlim(xlim)
        a.set_ylim(ylim)
    donut_roads.plot(ax=ax[0], color="white", zorder=5)
    donut_roads.plot(ax=ax[1], color="#7B73C4", zorder=5)
    
    mm.nx_to_gdf(graph, points=False).plot(ax=ax[0], color="#7B73C4", zorder=6)
    if max_betweeness:
        removals.append(max_node[:2])
        c = ax[0].scatter(x, y, c=c, s=[cc/max(c)*80 for cc in c], cmap="RdPu", zorder=7)
        ax[1].scatter(max_node[0], max_node[1], s=150, c="#D40058", zorder=7)
    
    ax[1].scatter(*zip(*removals), s=50, c="#D40058", zorder=6)
    ax[0].set_title("Betweenness Centrality")
    ax[1].set_title("Closures")
    
#     if not len([b for b in betweeness.values() if b > 0]):
#     break

    graph.remove_node(max_node)
    plt.savefig(f"fig_{i}.png")
#     plt.show()

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(15, 10), sharex=True, sharey=True)
roads.plot(ax=ax[0], color="lightgrey", zorder=1)
shell_gdf.plot(ax=ax[0], color="#a09ad5", alpha=0.2, zorder=1)
hole_gdf.plot(ax=ax[0], color="#D40058", alpha=0.2, zorder=1)
hole_roads.plot(ax=ax[0], color="#D40058", alpha=0.4, zorder=1)
donut_roads.plot(ax=ax[0], color="#7B73C4")
ax[0].scatter(centre[0], centre[1], c="#D40058", s=100, zorder=2)
ax[0].set_xlim(xlim)
ax[0].set_ylim(ylim)
plt.savefig(f"fig_empty.png")

f, ax = plt.subplots(1, 2, figsize=(15, 10), sharex=True, sharey=True)
roads.plot(ax=ax[0], color="lightgrey", zorder=1)
shell_gdf.plot(ax=ax[0], color="#a09ad5", alpha=0.2, zorder=1)
hole_gdf.plot(ax=ax[0], color="#D40058", alpha=0.2, zorder=1)
hole_roads.plot(ax=ax[0], color="#D40058", alpha=0.4, zorder=1)
donut_roads.plot(ax=ax[0], color="#7B73C4", zorder=1)
hole_points.plot(ax=ax[0], color="#D40058", zorder=2)
shell_points.plot(ax=ax[0], color="#7B73C4", zorder=2)
ax[0].scatter(centre[0], centre[1], c="#D40058", s=200, zorder=2)
ax[0].set_xlim(xlim)
ax[0].set_ylim(ylim)
plt.savefig(f"fig_nodes.png")